In [1]:
!python -V

Python 3.11.7


In [2]:
!pip install seaborn

In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import numpy as np

# Suponiendo que tienes y_true y y_pred definidos
# rmse = np.sqrt(mean_squared_error(y_true, y_pred))


In [26]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [29]:
#lectura del parquet
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)




In [30]:
duracion_ed = df['duration'].std()
print(duracion_ed)


42.594351241920904


In [33]:
filas_antes = len(df)
filas_antes

3066766

In [35]:
# aplicando el filtro


df = df[(df.duration >= 1) & (df.duration <= 60)]
filas_fil = len(df)
print(filas_fil)
"""
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)
"""

3009173


"\ncategorical = ['PULocationID', 'DOLocationID']\nnumerical = ['trip_distance']\n\ndf[categorical] = df[categorical].astype(str)\n"

In [38]:
var_per = (filas_antes-filas_fil)/filas_antes
print((1-var_per)*100)

98.1220282212598


In [39]:


# Set the categorical variables we'll consider
categorical = ['PULocationID', 'DOLocationID']

# Re-cast the ids to strings - otherwise it will label encode them
df[categorical] = df[categorical].astype(str)
# Turn to a lists of dictionaries
train_dicts = df[categorical].to_dict(orient='records')
# Create a DictVectorizer and fit the variables
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)



In [46]:
print(X_train.shape)

(3009173, 515)


In [49]:


# Extract the labels
target = 'duration'
y_train = df[target].values

# Create a linear regressor with default parameters
lr = LinearRegression()
# Train the regressor
lr.fit(X_train, y_train)
# Get the predictions
y_pred = lr.predict(X_train)
# Calculate the RSMEW metric for the actual labels

from sklearn.metrics import mean_squared_error
import numpy as np

# Suponiendo que tienes y_train (valores reales) y y_pred (valores predichos)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print("RMSE:", rmse)


RMSE: 7.649262055538092


In [59]:
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
df_val[categorical] = df_val[categorical].astype(str)

# Apply the same transformation that were applied to the training dataset
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)



print(X_val.shape)



(2855951, 515)


In [60]:
target = 'duration'
y_val = df_val[target].values
print("Length validation labels: ", y_val.shape)

Length validation labels:  (2855951,)


In [61]:
y_val

array([ 1.68333333, 32.08333333, 13.3       , ..., 14.        ,
        7.        ,  9.8       ])

In [62]:


# Get predictions for validation features
y_pred = lr.predict(X_val)






In [63]:
y_pred = lr.predict(X_val)
#º Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))


In [64]:
print(rmse)

7.811816386688629
